In [3]:
# 执行本脚本更新flags列表文件： iwencai_flags.json / iwencai_flags_withtype.json
# 写入文件之前对flag进行标准化：
# 去除“停牌”，去除所有“最近”开头的flag
# 将“今日”开头的flag的“今日”两字去除

from opentrader.core.crawler import *
from opentrader.agents.xueqiu.api import *
from opentrader.agents.ths.api import *
c = CNCrawler()
stock_list = c.get_stock_list()

Querying 54 pages ......................................................

In [4]:
import json, threading, sys
from datetime import datetime,timedelta
stockflags = [] # [{'symbol':'', 'buy':[], 'sell':[], 'event':[], 'zxst':[]}, ...]
lock = threading.RLock()
ths = THSAPI()

# create 10 threads to do the job
class StockAnalyzer(threading.Thread):
    def __init__(self, stocks):
        threading.Thread.__init__(self)
        self.stocks = stocks
        
    def run(self):
        # for each stock, get latest flags
        for i,stock in enumerate(self.stocks):
            #if (i > 100):
            #    break
            print('Querying stock %d, %s...' % (i, stock['symbol']))
            sys.stdout.flush()
            try:
                result = ths._get_stock_analyze(stock['symbol'])
            except InvalidSymbol:
                continue
            stockflag = {'symbol':stock['symbol'], 'buy':[], 'sale':[], 'event':[], 'zxst':[]}
            for key in ('event','buy','sale','zxst'):
                #print(result['data'])
                try:
                    flag_dict = result['data']['data']['result'][key]
                except (TypeError, KeyError):
                    continue
                for (k, flag_item) in flag_dict.items():
                    flag = flag_item['query']
                    stockflag[key].append(flag)
            lock.acquire()
            stockflags.append(stockflag)
            lock.release()

each_len = len(stock_list) / 10
threads = []
for i in range(100):
    stocks = stock_list[i*400:(i+1)*400]
    if len(stocks) == 0:
        break
    thread = StockAnalyzer(stocks)
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()

f = open('flags' + str(datetime.now().date())+'.json', 'w')
f.write(json.dumps(stockflags))

Querying stock 0, HKHSF...
Querying stock 1, SH603999...
Querying stock 0, SH600797...
Querying stock 0, SH600322...
Querying stock 0, SZ399682...
Querying stock 0, SZ300377...
Querying stock 0, SZ200413...
Querying stock 0, SH104709...
Querying stock 0, SH104020...
Querying stock 0, SZ002769...
Querying stock 0, SZ002364...
Querying stock 0, SZ000980...
Querying stock 0, SZ000599...
Querying stock 1, SH104019...
Querying stock 1, SZ399681...
Querying stock 1, SZ200168...
Querying stock 1, SH104708...
Querying stock 1, SZ300376...
Querying stock 1, SH000979...
Querying stock 1, SH600796...
Querying stock 1, SZ002363...
Querying stock 2, SH603998...
Querying stock 1, SZ000598...
Querying stock 2, SH104707...
Querying stock 2, SH104018...
Querying stock 2, SZ200160...
Querying stock 2, SZ399680...
Querying stock 2, SZ000979...
Querying stock 2, SZ000597...
Querying stock 2, SZ002362...
Querying stock 3, SH104706...
Querying stock 3, SZ200152...
Querying stock 3, SH000978...
Querying stoc

4162911

In [5]:
print(len(stock_list))
print(len(stockflags))

4725
4724


In [6]:
# make statistics over all flags and update to file "iwencai_flags.json"
try:
    f = open('iwencai_flags.json', 'r')
    flags = json.loads(f.read())
    print('total flags: %d' % len(flags))
    f.close()
except:
    flags = []

flags_withtype = {'buy':[], 'sale':[], 'event':[], 'zxst':[]}
for each in stockflags:
    for key in ('buy', 'sale', 'event', 'zxst'):
        for each_flag in each[key]:
            # 标准化flag
            excludes = ("停牌", "最近", "持续", "连阴", "连阳", "连涨", "连跌", "持股")
            exc = False
            for each_ex in excludes:
                if each_ex in each_flag:
                    exc = True
                    break
            if exc:
                continue
            if "上穿5日" in each_flag or "下穿5日" in each_flag:
                each_flag = each_flag + "均线"
            if each_flag.startswith("今日"):
                each_flag = each_flag[2:]
            if each_flag not in flags:
                flags.append(each_flag)
                print('Added flag: %s' % each_flag)
            if each_flag not in flags_withtype[key]:
                flags_withtype[key].append(each_flag)
print('---------After Update----------')
print('total flags: %d' % len(flags))
f = open('iwencai_flags.json', 'w')
f.write(json.dumps(flags))
f.close()
f = open('iwencai_flags_withtype.json', 'w')
f.write(json.dumps(flags_withtype))
f.close()
print(flags_withtype)

total flags: 555
Added flag: 月线wr超卖
Added flag: wr超卖
Added flag: 周线wr超卖
Added flag: 金针探底
Added flag: 周线倒v型反转
Added flag: 股价创30日新低
Added flag: 周线boll跌破下轨
Added flag: 周线价跌量平
Added flag: 周线wr超买
Added flag: 突破布林线上轨
Added flag: 周线巨量长阳
Added flag: 回踩通道线
Added flag: wr超买
Added flag: boll跌破下轨
Added flag: 股价跌破布林线下轨
Added flag: 周线双飞乌鸦
Added flag: 月线光脚阴线
Added flag: 月线光头光脚阴线
Added flag: KDJ J线低于10
Added flag: 资金净流入连续3天〉0
Added flag: 周线rsi底背离
Added flag: psy超卖
Added flag: 倒v型反转
Added flag: 周线价平量升
Added flag: 月线dmi死叉
Added flag: 周线金针探底
Added flag: 月线wr超买
Added flag: 周线两黑夹一红
Added flag: 周线psy超卖
Added flag: 月线boll跌破下轨
Added flag: 周线锤子线
Added flag: 月线锤子线
Added flag: 周线放量突破
Added flag: 阳包阴股票
Added flag: 月线金针探底
Added flag: macd日线指标背离
Added flag: 周线底部放量
Added flag: 最新收盘价在60日均线上的
Added flag: 周线刺穿线
Added flag: 连续5天下跌主力资金进入
Added flag: 双针探底
Added flag: 月线大s均线
Added flag: 周线季价托
Added flag: 月线长阳重炮
Added flag: 倒t字线
Added flag: 周线倒t字线
Added flag: 月线倒t字线
---------After Update----------
total flags: 602
{'event':